In [1]:
import pandas as pd
import numpy as np

# 1 - Obtenga el dataset “data_ventas”.

In [2]:
ventas = pd.read_excel('data_ventas.xlsx') # Importamos los datos para utilizarlos

Inspeccionamos los datos a utilizar y algunas propiedades estadisticas del mismo

In [3]:
ventas.head() # Inspeccionamos los primeros elementos del dataset

,IdVenta,Fecha,Cantidad,Monto,Producto,Color,Departamento,Revenue,Barrio
0,1,2012-03-30,16.0,1600.0,Puerta,Negro,Montevideo,25600.0,1
1,2,2014-06-16,120.0,2400.0,Ventana,Negro,Durazno,288000.0,Sin datos
2,3,2014-08-20,61.0,520.0,Puerta,Negro,Paysandú,31720.0,Sin datos
3,4,2010-02-16,4.0,780.0,Hazada,Rojo,Salto,3120.0,Sin datos
4,5,2009-06-11,16.0,520.0,"Chapa 3,2",Negro,Cerro Largo,8320.0,Sin datos


In [4]:
ventas.info() # Observamos tipos de datos de las columnas y cantidad de valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   IdVenta       100000 non-null  int64         
 1   Fecha         100000 non-null  datetime64[ns]
 2   Cantidad      99988 non-null   float64       
 3   Monto         99977 non-null   float64       
 4   Producto      100000 non-null  object        
 5   Color         100000 non-null  object        
 6   Departamento  100000 non-null  object        
 7   Revenue       100000 non-null  float64       
 8   Barrio        100000 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 6.9+ MB


In [5]:
# Analizamos parametros de la población analizada para tener una idea sobre los ordenes de magnitud y outliers
ventas.describe()

,IdVenta,Fecha,Cantidad,Monto,Revenue
count,100000.000000,100000,99988.000000,99977.000000,1.000000e+05
mean,50000.500000,2011-01-23 00:40:36.480000,72.891747,675.288016,5.480474e+04
min,1.000000,2007-01-01 00:00:00,1.000000,260.000000,0.000000e+00
25%,25000.750000,2009-01-09 00:00:00,31.000000,387.000000,1.534000e+04
50%,50000.500000,2011-01-28 00:00:00,62.000000,400.000000,3.042000e+04
75%,75000.250000,2013-02-04 00:00:00,93.000000,800.000000,5.280000e+04
max,100000.000000,2015-02-18 00:00:00,1200.000000,4800.000000,5.760000e+06
std,28867.657797,NaN,102.278427,448.049767,1.455684e+05


## a. Filtre el dataset de modo de trabajar solo con datos de Montevideo


In [6]:
# Utilizamos el comando '.loc[]' para filtrar los datos que nos interesan
ventas = ventas.loc[(ventas['Departamento'] == 'Montevideo')]
ventas.head()

,IdVenta,Fecha,Cantidad,Monto,Producto,Color,Departamento,Revenue,Barrio
0,1,2012-03-30,16.0,1600.0,Puerta,Negro,Montevideo,25600.0,1
15,16,2010-05-28,118.0,780.0,"Chapa 2,3",Negro,Montevideo,92040.0,53
27,28,2012-10-27,69.0,2400.0,Hazada,Verde,Montevideo,165600.0,21
53,54,2009-06-22,1190.0,1200.0,Pala Pocera,Negro,Montevideo,1428000.0,47
55,56,2013-01-25,570.0,2400.0,Hazada,Rojo,Montevideo,1368000.0,78


## b. Elimine la columna “Color”

In [9]:
# Eliminamos la columna 'Color' debido a que no nos interesa en el análisis
ventas = ventas.drop(columns = 'Color')
#del ventas["Color"]
ventas.head()

KeyError: "['Color'] not found in axis"

## c. Identifique los valores faltantes (no necesariamente son valores nulos)

Aquí comienza el trabajo más arduo del analista de datos, analizar los datos que tiene con rigurosidad. Para comenzar podríamos utilizar algún comando para observar valores nulos.

### I. Valores numericos, reemplace los faltantes por la media del valor de la columna.


In [10]:
# Utilizamos este comando para revisar que columnas son valores numericos
ventas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7090 entries, 0 to 99987
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IdVenta       7090 non-null   int64         
 1   Fecha         7090 non-null   datetime64[ns]
 2   Cantidad      7078 non-null   float64       
 3   Monto         7067 non-null   float64       
 4   Producto      7090 non-null   object        
 5   Departamento  7090 non-null   object        
 6   Revenue       7090 non-null   float64       
 7   Barrio        7090 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 498.5+ KB


**Podemos observar que solo existen 4 columnas con valores numericos. La IdVenta no es tan importante para analizar valores nulos, ya que es un entero auto-incremental.**

In [11]:
ventas.isnull().sum() # Observamos los valores que son nulos en el dataset

IdVenta          0
Fecha            0
Cantidad        12
Monto           23
Producto         0
Departamento     0
Revenue          0
Barrio           0
dtype: int64

In [12]:
# Analizamos que sucede cuando las columnas 'Cantidad' y 'Monto' son nulas
ventas.loc[(ventas['Cantidad'].isnull()) | (ventas['Monto'].isnull())] # Recordar que el simbolo "|" implica "or"

,IdVenta,Fecha,Cantidad,Monto,Producto,Departamento,Revenue,Barrio
87,88,2012-08-04,790.0,NaN,Pala hogar,Montevideo,0.0,13
193,194,2011-09-17,1140.0,NaN,Pico,Montevideo,0.0,17
219,220,2014-06-02,70.0,NaN,Pico,Montevideo,0.0,15
29829,29830,2014-02-13,NaN,800.0,Rastrillo,Montevideo,0.0,37
29831,29832,2014-01-20,20.0,NaN,Pico,Montevideo,0.0,38
29893,29894,2007-06-26,71.0,NaN,Pala hogar,Montevideo,0.0,10
29922,29923,2009-12-11,NaN,800.0,Puerta,Montevideo,0.0,43
29985,29986,2013-09-28,11.0,NaN,Pala hogar,Montevideo,0.0,30
30039,30040,2014-12-31,NaN,NaN,Puerta,Montevideo,0.0,71
30098,30099,2009-03-22,NaN,800.0,Puerta,Montevideo,0.0,43


**Notese el comportamiento de la columna 'Revenue' cuando la cantidad o el monto son valores nulos. Analizamos todos los valores donde Revenue == 0**

In [20]:
ventas.loc[ventas['Revenue'] == 0] # Vemos que la columna Revenue == 0 SOLO cuando cantidad o monto son NaN.

,IdVenta,Fecha,Cantidad,Monto,Producto,Departamento,Revenue,Barrio


In [19]:
ventas.loc[(ventas['Cantidad'] == 0) | (ventas['Monto'] == 0)] # No existen valores iguales a 0 para las columnas Cantidad y Monto

,IdVenta,Fecha,Cantidad,Monto,Producto,Departamento,Revenue,Barrio


In [15]:
media_cantidad = ventas['Cantidad'].mean(skipna = True)
ventas.fillna({'Cantidad': media_cantidad}, inplace = True)
media_monto = ventas['Monto'].mean(skipna = True)
ventas.fillna({'Monto': media_monto}, inplace = True)

# ¿Se puede mejorar esto?
# ¿Que hacemos con la columna Revenue?
# ¿Hay algún outlier?
# ¿Debería preocuparme por los duplicados?

In [18]:
# Al analizar la columna Revenue = Monto * Cantidad
ventas['Revenue'] = ventas['Monto'] * ventas['Cantidad']

In [21]:
# ¿Se puede mejorar esto? -> Si, podríamos hacer la media de cada producto, debido a que son productos distintos
for columna in ventas['Producto'].value_counts().index:
  print(columna)

Pico
Puerta
Pala hogar
Ventana
Pala Pocera
Rastrillo
Pala Dientes
Chapa 3,2
Hazada
Chapa 2,3


In [31]:
# Hay algún outlier?
ventas.describe()
#ventas.loc[(ventas["Cantidad"]==1)].value_counts()

,IdVenta,Fecha,Cantidad,Monto,Revenue
count,7090.000000,7090,7090.000000,7090.000000,7.090000e+03
mean,43777.653315,2011-01-21 18:23:51.875881472,70.580390,676.923235,5.314955e+04
min,1.000000,2007-01-01 00:00:00,1.000000,260.000000,2.600000e+02
25%,19888.000000,2008-12-29 06:00:00,31.000000,390.000000,1.534000e+04
50%,36917.500000,2011-01-30 12:00:00,61.000000,400.000000,3.058650e+04
75%,67868.500000,2013-01-26 18:00:00,92.000000,800.000000,5.100000e+04
max,99988.000000,2015-02-18 00:00:00,1200.000000,4800.000000,5.376000e+06
std,28466.269313,NaN,97.666722,456.037382,1.610991e+05


In [32]:
ventas.loc[(ventas['Monto'] == 4800) | (ventas['Cantidad'] == 1200)] # Identifican algún patron?

,IdVenta,Fecha,Cantidad,Monto,Producto,Departamento,Revenue,Barrio
73,74,2008-03-22,1120.0,4800.0,Pala hogar,Montevideo,5376000.0,76
161,162,2009-04-03,890.0,4800.0,Hazada,Montevideo,4272000.0,73
213,214,2009-06-27,340.0,4800.0,Hazada,Montevideo,1632000.0,57
225,226,2011-10-16,1020.0,4800.0,Rastrillo,Montevideo,4896000.0,57
1985,1986,2012-12-27,1200.0,900.0,"Chapa 3,2",Montevideo,1080000.0,81


### II. Cuando sean categóricos, elimínelos. 

In [33]:
for columna in ventas.columns:
  print(ventas[columna].value_counts()) # Vemos que valores tienen las columnas categóricas

IdVenta
1        1
56338    1
56488    1
56464    1
56444    1
        ..
25339    1
25328    1
25321    1
25319    1
99988    1
Name: count, Length: 7090, dtype: int64
Fecha
2008-03-11    9
2015-02-12    8
2012-01-19    8
2008-01-08    8
2007-11-19    8
             ..
2011-09-21    1
2007-10-14    1
2009-08-21    1
2007-02-10    1
2012-04-08    1
Name: count, Length: 2730, dtype: int64
Cantidad
19.0     83
47.0     83
93.0     79
45.0     76
30.0     73
         ..
200.0     1
580.0     1
700.0     1
320.0     1
470.0     1
Name: count, Length: 190, dtype: int64
Monto
400.000000     1234
600.000000      958
800.000000      924
1200.000000     656
390.000000      638
345.000000      622
1600.000000     597
387.000000      597
260.000000      546
2400.000000      64
900.000000       45
585.000000       25
517.500000       24
676.923235       23
580.500000       23
690.000000       21
774.000000       19
1800.000000      19
780.000000       17
520.000000       14
3200.000000      12
360

In [ ]:
ventas.loc[(ventas['Barrio'] == 'Sin datos')] # Nos fijamos el DataFrame cuando la columna Barrio == Sin datos, para buscar patrones

In [ ]:
#ventas['Barrio'] = ventas['Barrio'].replace('Sin datos', np.nan) # Reemplazo textual
# o
ventas.loc[ventas['Barrio'].astype('str').str.contains('sin datos', case=False, na= False), 'Barrio'] = np.nan # Reemplazo 'like'

In [ ]:
ventas = ventas.dropna(subset=['Barrio']) # Elimino las filas con valores NaN
ventas.head()

In [ ]:
ventas.loc[(ventas['Barrio'].isnull())] # Me fijo si fueron borrados

## e. Obtenga el dataset “barrios_mdeo”

In [ ]:
barrios = pd.read_csv('barrios_mdeo.csv', delimiter = ';',encoding = 'latin-1') # Observo un error de UTF-8 y luego de delimitador
barrios.head()

### I. Combine el dataset con data_ventas, teniendo éste último a la izquierda en un left-join.

In [ ]:
barrios.rename(columns={'código':'Barrio'}, inplace=True) # Es más sencillo para agrupar luego

In [ ]:
montevideo = ventas.merge(barrios, on='Barrio', how='left') # left-join utilizando la columna 'Barrio' como referencia
montevideo.head()

### II. Elimine las variables que muestran el código de los barrios, quedándose solo con los nombres.
product


In [ ]:
montevideo.drop(columns='Barrio', inplace=True) # De no renombrar la columna 'código' deberian eliminarse dos columnas
montevideo.head()

### iii. Haga un conteo de valores por barrio.

Investigue la forma de exportar a un Excel el conteo

In [ ]:
montevideo['Lugar'].value_counts().to_excel('Conteo_barrios.xlsx')

# 2. Obtenga el dataset “registros.xlsx”. El Excel tiene registros por ventas para dos meses, obtenga ambos datasets.

In [ ]:
registros_1 = pd.read_excel('registros.xlsx', sheet_name='mes1')
registros_2 = pd.read_excel('registros.xlsx', sheet_name='mes2')

## a. Genere una dataset final “registros” que sea la anexión de ambos archivos. 

In [ ]:
registros = pd.concat([registros_1,registros_2], ignore_index=True)
registros

## b. La columna Id debe ser el index, asegúrese que éste no tenga duplicados.

In [ ]:
registros['id'].value_counts() # Vemos que la columna id tiene valores duplicados

In [ ]:
registros.drop(columns=['id'], inplace=True) # En este caso si no se utiliza en bases de datos relacionales, es mejor eliminarla

In [ ]:
registros.head()

## Se necesita obtener el precio de los productos en dólares a valor actual, por tanto, utilice la función
read_html (ejemplo), para obtener este dato directamente de la página de cotizaciones del BCU, y
genere una columna nueva valor_dolares, utilizando el número obtenido1

In [ ]:
url = 'https://www.bcu.gub.uy/Estadisticas-e-Indicadores/Paginas/Cotizaciones.aspx'
cotizaciones = pd.read_html(url, thousands = '.')
cotizacion = cotizaciones[4]

In [ ]:
valor_dolar = cotizacion.loc[0,2]
valor_dolar = valor_dolar.replace(',', '.')

In [ ]:
registros['valor_dolares'] = registros['value'] / float(valor_dolar)

In [ ]:
registros